# Homework 2

## 데이터 준비

In [1283]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

import torch, random; torch.manual_seed(42); random.seed(42) # 시드 고정

# 출력 폭 제한을 해제 (무제한)
pd.set_option("display.width", None)
# 모든 컬럼 출력되도록 설정
pd.set_option("display.max_columns", None)

def get_base_dir():는 주피터 노트북 환경에서는 Path(__file__).resolve().parent이 존재하지 않기 때문에 동일한 Path.cwd() 으로 대체하는 함수이다.

sys.path.append(BASE_PATH) 은 코드가 살펴보는 라이브러리 경로에 임의 경로를 추가하여 임포트를 편리하게 한다.

In [1284]:
from pathlib import Path

# path 초기화 (아마도)
sys.path = [(p if isinstance(p, str) else os.fspath(p))
            for p in sys.path
            if isinstance(p, str) or hasattr(p, "__fspath__")]

def get_base_dir():
    try:
        return Path(__file__).resolve().parent  # .py 스크립트/모듈일 때
    except NameError:
        return Path.cwd()                       # 노트북/인터랙티브일 때
    
BASE_PATH = get_base_dir().parent # BASE_PATH: /Users/yhhan/git/link_dl
import sys
sys.path.append(BASE_PATH)

from _01_code._99_common_utils.early_stopping import EarlyStopping # Early Stopping

Dataset을 상속하는 커스텀 데이터셋 클래스에는 __len__()와 __getitem__(1)가 필수로 들어간다.
이것을 오버로딩해줘야 데이터셋을 로드하거나 스플릿 할 수 있다.

In [1285]:
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

In [1286]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

get_preprocessed_dataset_N 함수들은 라이브러리 기능을 활용하여 데이터셋 전처리를 하는 부분이다.

In [1287]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare (요금) 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    all_df = all_df.drop(columns=["Fare_mean"])

    return all_df

In [1288]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "title", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["title"] = name_df["title"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

In [1289]:
def get_preprocessed_dataset_3(all_df):
    # title별 Age 평균값을 사용하여 Age 결측치 메우기
    title_age_mean = all_df[["title", "Age"]].groupby("title").median().round().reset_index()
    title_age_mean.columns = ["title", "title_age_mean", ]
    all_df = pd.merge(all_df, title_age_mean, on="title", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["title_age_mean"]
    all_df = all_df.drop(["title_age_mean"], axis=1)

    return all_df

In [1290]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1

    #all_df["alone"].fillna(0, inplace=True) # 경고 때문에 고침
    all_df["alone"]    = all_df["alone"].fillna(0)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [1291]:
def get_preprocessed_dataset_5(all_df):
    # title 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["title"] == "Mr") |
            (all_df["title"] == "Miss") |
            (all_df["title"] == "Mrs") |
            (all_df["title"] == "Master")
    ),
    "title"
    ] = "other"
    #all_df["Embarked"].fillna("missing", inplace=True) # 경고 때문에 고침
    all_df["Embarked"] = all_df["Embarked"].fillna("missing")

    return all_df

In [1292]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

pd.read_csv() 함수로 경로에 있는 csv를 읽어 판다스 형태로 읽어오고 전처리한 판다스 형태의 자료를 학습용, 검증용, 시험용으로 나눠 반환한다.

In [1293]:
def get_preprocessed_dataset():
    #CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
    CURRENT_FILE_PATH = get_base_dir()

    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    print(all_df.columns)
    print(all_df.head(10))

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    dataset = TitanicDataset(train_X.values, train_y.values)
    print(dataset)

    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

전처리 확인용 코드

In [1294]:
if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset), shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[TEST]")
  for idx, batch in enumerate(test_data_loader):
    print("{0} - {1}".format(idx, batch['input'].shape))

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

## 학습 루프

In [1295]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse

CURRENT_FILE_PATH = os.path.dirname(get_base_dir())
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

In [1296]:
def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print(len(train_dataset), len(validation_dataset))

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset), shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  return train_data_loader, validation_data_loader

FCN을 활용한 모델을 만든다. nn.Module을 상속받고 super().__init__()으로 상속받은 클래스의 생성자를 실행시킨다.
nn.Sequential을 사용하여 연속적인 레이어를 쌓는다. forward 함수를 오버로딩해주어야 순전파를 시행할 수 있다.

In [1297]:
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.Sigmoid(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.Sigmoid(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
      nn.Sigmoid(),
    )

  def forward(self, x):
    x = self.model(x)
    return x

optim.SGD 으로 파라미터들의 역전파시 어떤 방식(옵티마이저)으로 경사하강법을 시행할 것인지 설정한다. 여기서는 확률적 경사 하강법을 사용한다.

In [1298]:
def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=1)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

In [1299]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.BCELoss()
  next_print_epoch = 100

  early_stopping = EarlyStopping(
    patience=wandb.config.early_stop_patience,
    delta=wandb.config.early_stop_delta,
    project_name="",
    checkpoint_file_path=CHECKPOINT_FILE_PATH,
    run_time_str=datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
  )

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input, target = train_batch["input"].float(), train_batch["target"].float()
      if target.ndim == 1:              # ← 추가
        target = target.unsqueeze(1)       # ← 추가
      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input, target = validation_batch["input"].float(), validation_batch["target"].float()
        if target.ndim == 1:              # ← 추가
          target = target.unsqueeze(1)       # ← 추가
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    validation_loss = loss_validation / num_validations

    if epoch > n_epochs*0.05:
      message, early_stop = early_stopping.check_and_save(validation_loss, model)
    else:
      message, early_stop = "", False

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
        f"{message} | "
      )
      next_print_epoch += 100

    
    #if early_stop:
      #break

In [1300]:
def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'early_stop_patience': 100,
    'early_stop_delta': 0,
    'validation_intervals': 100,
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "homework2"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish()

  # --- test dataset 준비
  _, _, test_dataset = get_preprocessed_dataset()
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset))

  # --- 예측
  with torch.no_grad():
      for batch in test_loader:
          inputs = batch['input']
          outputs = linear_model(inputs)
          preds = (outputs >= 0.5).int()
          preds = preds.ravel().int()

  # --- submission.csv 파일로 저장
  submission = pd.DataFrame({
      'PassengerId': range(892, 892 + len(preds)),  # Kaggle Titanic 기준 test.csv ID 시작 892
      'Survived': preds
  })
  submission.to_csv('submission.csv', index=False)

  print("✅ submission.csv 저장 완료:", submission.head())

In [1301]:
# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=True, help="True or False"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=10_000, help="Number of training epochs (int, default:1_000)"
  )

  args = parser.parse_args(args=[])
  
  main(args)

Epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
Training loss,█▄▄▄▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Validation loss,▅▃▃▂▂▂▁▁▂▂▁▁▁▂▁▂▂▂▃▃▃▄▃▃▃▅▄▄▄▅▅▆▇▆▆▇█▆▇█
Epoch,1327
Training loss,0.25323
Validation loss,0.64952


Namespace(wandb=True, batch_size=16, epochs=10000)
{'epochs': 10000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20], 'early_stop_patience': 100, 'early_stop_delta': 0, 'validation_intervals': 100}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3     

Epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇████
Training loss,██▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▄▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation loss,██▇▇▇▆▆▆▆▆▅▅▅▅▅▄▃▃▂▂▁▂▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁
Epoch,10000
Training loss,0.39603
Validation loss,0.4647


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

## [요구사항 1] titanic 딥러닝 모델 기본 훈련

![](https://github.com/Minelauncher/LINK_DL-SAVE/blob/main/HW2/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89%20%EC%BA%90%EA%B8%80%20%EA%B2%B0%EA%B3%BC.png?raw=1)

## [요구사항 2] Activation Function과 Batch Size변경 및 선택하기

![](https://github.com/Minelauncher/LINK_DL-SAVE/blob/main/HW2/%ED%99%9C%EC%84%B1%ED%99%94%20%ED%95%A8%EC%88%98%20%EB%B9%84%EA%B5%90%20%EC%8B%9C%EB%93%9C%20%EA%B3%A0%EC%A0%95.png?raw=1)

캐글에 제출한 결과를 바탕으로 Sigmoid, ELU, LeakyReLU, ReLU중 Sigmoid가 가장 분류 성능이 좋았음을 알 수 있었다.

하이퍼 파라미터는 1만 에포크를 기준으로 배치사이즈를 16으로 고정하여 시도하였다.

![](https://github.com/Minelauncher/LINK_DL-SAVE/blob/main/HW2/%EB%B0%B0%EC%B9%98%20%EC%82%AC%EC%9D%B4%EC%A6%88%20%EB%B9%84%EA%B5%90%20%EC%8B%9C%EB%93%9C%20%EA%B3%A0%EC%A0%95.png?raw=1)

캐글에 제출한 결과를 바탕으로 16, 32, 64, 128 배치 사이즈를 비교하였다. 배치 사이즈 16이 가장 성능이 좋았고 배치 사이즈가 커질 수록 정확도가 낮아짐을 확인 할 수 있었다.

## [요구사항 3] 테스트 및 submission.csv 생성

In [1302]:
config = {
'n_hidden_unit_list': [20, 20],
}

wandb.init(
mode="online" if args.wandb else "disabled",
project="my_model_training",
notes="My first wandb experiment",
tags=["my_model", "homework2"],
config=config
)

latest_model = MyModel(n_input=10, n_output=1)
latest_model.eval() # 모델 평가 모드
latest_file_path = os.path.join(CHECKPOINT_FILE_PATH, "_checkpoint_latest.pt")
latest_model.load_state_dict(torch.load(latest_file_path, map_location=torch.device('cpu')))

# --- test dataset 준비
_, _, test_dataset = get_preprocessed_dataset()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset))

# --- 예측
with torch.no_grad():
    for batch in test_loader:
        inputs = batch['input']
        outputs = latest_model(inputs)
        preds = (outputs >= 0.5).int()
        preds = preds.ravel().int()

# --- submission.csv 파일로 저장
latest_submission = pd.DataFrame({
    'PassengerId': range(892, 892 + len(preds)),  # Kaggle Titanic 기준 test.csv ID 시작 892
    'Survived': preds
})
latest_submission.to_csv('latest_submission.csv', index=False)

print("✅ latest_submission.csv 저장 완료:", latest_submission.head())

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

![](https://github.com/Minelauncher/LINK_DL-SAVE/blob/main/HW2/Early%20Stopping%20%EA%B2%80%EC%A6%9D.png?raw=1)

## [요구사항 4] submission.csv 제출 및 등수확인

![](https://github.com/Minelauncher/LINK_DL-SAVE/blob/main/HW2/%EC%BA%90%EA%B8%80%20%EB%93%B1%EC%88%98.png?raw=1)

## 숙제 후기

파이토치의 코드를 구경할 수 있어서 좋았다.

하지만 경로 문제가 너무 걸렸고 주피터 노트북과 같은 형태로 제출하는 것이 힘들다. 기존 코드를 수정해야 했기 때문이다.

코드 전처리 부분에서는 판다스 문법을 다 까먹어서 이해가 어려웠다.

Adam 옵티마이저를 쓰면 조금은 그래프가 깔끔해졌다.

WanDB같은 좋은 시각화 사이트가 있다는 것을 처음 알았다.